1. https://codedragon.tistory.com/9556 해당 사이트 참고하여 설치 의존성 문제때문에 whl 파일로 순차적으로 다운로드

2. https://datascienceschool.net/view-notebook/ef921dc25e01437b9b5c532ba3b89b02/ 예제 코드 참고
3. https://github.com/rollinstar/gpd_example 예제 참고
4. https://datascienceschool.net/view-notebook/ef921dc25e01437b9b5c532ba3b89b02/#Geoplot
5. https://medium.com/@thlee33/%EC%9C%88%EB%8F%84%EC%9A%B0-%EA%B8%B0%EB%B0%98-geopandas-%EC%84%A4%EC%B9%98-%EB%B0%8F-%EC%A2%8C%ED%91%9C-%EB%B3%80%ED%99%98-e9b9ef16d9f9
6. https://geopandas.org/mapping.html

# 개념
1. geometry 라는 컬럼은 여러 개의 point로 나타나져 있다.

# 기본 연산
1. within 지리적으로 포함 '되는지' 에 대한 여부
2. contains 지리적으로 포함 '하는지' 에 대한 여부
3. intersects 지리적으로 교차하는지 여부, 두 지리가 경계선만 닫아 있어도 True 반환 (경계만 닿아있어도 True)
4. crosses : 지리적으로 교차하는 지 여부 crosees 는 내부를 지나가야 True 를 반환
5. squeeze() 메서드는 Geopandas 객체에서 Geometry 데이터만을 추출할 수 있는 기능을 한다.

# Geometry 데이터 속성
1. area: 넓이
2. boundary: 테두리
3. centroid: 중앙지점


필요 데이터

0. 서울시 격자 데이터 within 연산으로 격자당 설치 지수를 산출할 것
1. 전기차 현황
2. 인구 수에 대한 값을 어떻게 볼 수 있는가
3. 전기차 충전소 현황



In [1]:
import geopandas as gpd
import pandas as pd
from tqdm import tqdm
gpd.__version__

'0.8.0'

In [63]:
## Shape 파일 호출 하기

seoul_file = '격자_파일\총인구_전체.shp'
seoul = gpd.read_file(seoul_file, encoding = 'euc-kr')
print() # 이렇게 할 경우 값 산출

In [ ]:
## 결측치 검사 후 제거

seoul = seoul.to_crs(epsg=4326) # 좌표계 변환하기 4326으로 변환시 정상적으로 (우리가 아는) 좌표계 값 산출
seoul = seoul.dropna(axis=0) # Nan 에 대한 결측치 제거
seoul = seoul[seoul['lbl'] != 'N/A'] # N/A 에 대한 결측치 제거
seoul = seoul.drop('lbl',1) # val는 쓰지 않으므로 삭제 하여 필요한 Index만 남김
print(seoul.info()) # seoul geometry에 대한 info 출력
seoul.reset_index(drop =False, inplace = True)

In [81]:
seoul['car'] = 0
seoul['car_charge'] = 0
print(seoul.isnull().sum()) 
seoul

index         0
gid           0
val           0
geometry      0
car           0
car_charge    0
dtype: int64


,index,gid,val,geometry,car,car_charge
0,894,다사380537,14.0,"POLYGON ((126.79781 37.58060, 126.79780 37.581...",0,0
1,895,다사380538,31.0,"POLYGON ((126.79780 37.58150, 126.79780 37.582...",0,0
2,896,다사380539,7.0,"POLYGON ((126.79780 37.58240, 126.79779 37.583...",0,0
3,897,다사380540,12.0,"POLYGON ((126.79779 37.58330, 126.79778 37.584...",0,0
4,898,다사380541,15.0,"POLYGON ((126.79778 37.58420, 126.79777 37.585...",0,0
...,...,...,...,...,...,...
29334,61612,다사717517,195.0,"POLYGON ((127.17955 37.56423, 127.17955 37.565...",0,0
29335,61628,다사718511,315.0,"POLYGON ((127.18071 37.55882, 127.18071 37.559...",0,0
29336,61629,다사718512,442.0,"POLYGON ((127.18071 37.55972, 127.18070 37.560...",0,0
29337,61630,다사718513,426.0,"POLYGON ((127.18070 37.56062, 127.18070 37.561...",0,0


In [67]:
car = pd.read_excel('자동차등록현황.xlsx')
car_gdf = gpd.GeoDataFrame(car, geometry=gpd.points_from_xy(car['경도'], car['위도']))
car_gdf.crs = {'init': 'epsg:4326', 'no_defs': True}
car_gdf = car_gdf.to_crs(epsg=4326)
car_gdf['동별 총 대수'] = car_gdf['동별 총 대수'].fillna(0) ## 결측치에 대해서 0으로 채웠음
print(car_gdf.crs)
car_gdf

epsg:4326


C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,index,사용본거지법정동명,동별 총 대수,전기,하이브리드\n(휘발유+전기),하이브리드\n(경유+전기),하이브리드\n(LPG+전기),하이브리드\n(CNG+전기),위도,경도,geometry
0,0,서울특별시 종로구 청운동,32,4.0,27.0,NaN,1.0,NaN,37.589197,126.969330,POINT (126.96933 37.58920)
1,1,서울특별시 종로구 신교동,12,2.0,10.0,NaN,NaN,NaN,37.584490,126.967923,POINT (126.96792 37.58449)
2,2,서울특별시 종로구 궁정동,2,NaN,2.0,NaN,NaN,NaN,37.584683,126.973145,POINT (126.97314 37.58468)
3,3,서울특별시 종로구 효자동,7,1.0,6.0,NaN,NaN,NaN,37.582650,126.971932,POINT (126.97193 37.58265)
4,4,서울특별시 종로구 창성동,5,NaN,5.0,NaN,NaN,NaN,37.580545,126.972462,POINT (126.97246 37.58055)
...,...,...,...,...,...,...,...,...,...,...,...
466,466,서울특별시 강동구 둔촌동,247,15.0,225.0,NaN,7.0,NaN,37.527188,127.142355,POINT (127.14235 37.52719)
467,467,서울특별시 강동구 암사동,558,19.0,520.0,NaN,19.0,NaN,37.559114,127.131686,POINT (127.13169 37.55911)
468,468,서울특별시 강동구 성내동,530,59.0,464.0,1.0,6.0,NaN,37.529275,127.130027,POINT (127.13003 37.52927)
469,469,서울특별시 강동구 천호동,565,20.0,529.0,NaN,16.0,NaN,37.542222,127.128669,POINT (127.12867 37.54222)


In [116]:
# 점 찍은 것에 대해서 부풀려줘야함 -> 즉 동의 중심 찍은 것을 polygon 으로 만들어야함

seoul_area_file = 'tl_emd_seoul_4326/tl_emd._seoul_4326.shp'
seoul_area = gpd.read_file(seoul_area_file, encoding = 'euc-kr')
print() # 이렇게 할 경우 값 산출
seoul_area = seoul_area.to_crs(epsg=4326) # 좌표계 변환하기 4326으로 변환시 정상적으로 (우리가 아는) 좌표계 값 산출
print(seoul_area.info()) # seoul geometry에 대한 info 출력
print(seoul_area.crs)
seoul_area['car'] = 0
seoul_area



<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   emd_cd      467 non-null    object  
 1   emd_eng_nm  467 non-null    object  
 2   emd_kor_nm  467 non-null    object  
 3   geometry    467 non-null    geometry
dtypes: geometry(1), object(3)
memory usage: 14.7+ KB
None
epsg:4326


,emd_cd,emd_eng_nm,emd_kor_nm,geometry,car
0,11110152,Bongik-dong,봉익동,"POLYGON ((126.99254 37.57385, 126.99409 37.572...",0
1,11110153,Donui-dong,돈의동,"POLYGON ((126.99101 37.57336, 126.99178 37.570...",0
2,11560123,Mullae-dong 5(o)-ga,문래동5가,"POLYGON ((126.88453 37.51788, 126.88881 37.517...",0
3,11110141,Anguk-dong,안국동,"POLYGON ((126.98377 37.57947, 126.98426 37.579...",0
4,11110142,Sogyeok-dong,소격동,"POLYGON ((126.98112 37.58153, 126.98234 37.579...",0
...,...,...,...,...,...
462,11110145,Songhyeon-dong,송현동,"POLYGON ((126.98212 37.57863, 126.98258 37.576...",0
463,11290118,Dongseon-dong 3(sam)-ga,동선동3가,"POLYGON ((127.01958 37.59154, 127.01962 37.593...",0
464,11110123,Seorin-dong,서린동,"POLYGON ((126.98305 37.56998, 126.98285 37.568...",0
465,11110127,Gongpyeong-dong,공평동,"POLYGON ((126.98231 37.57202, 126.98429 37.572...",0


In [124]:
# 동에 대해 격자가 속하면 이를 seoul에 넣어줌
# 동 별 자동차 개수가 row가 된다.
# 그리고 격자가 동에 속하면 이 값을 격자에 넣어주면 된다. 
for i, rows in tqdm(car_gdf.iterrows(),total = car_gdf.shape[0]) : # 동 별 차량 대수
    seoul_area.loc[(seoul_area.geometry).contains(rows.geometry),'car'] = rows['동별 총 대수'] + seoul_area['car']



  0%|                                                                                          | 0/471 [00:00<?, ?it/s]

  4%|███                                                                             | 18/471 [00:00<00:02, 176.94it/s]

  7%|█████▉                                                                          | 35/471 [00:00<00:02, 173.34it/s]

 10%|████████▎                                                                       | 49/471 [00:00<00:02, 159.18it/s]

 15%|███████████▋                                                                    | 69/471 [00:00<00:02, 167.55it/s]

 18%|██████████████▎                                                                 | 84/471 [00:00<00:02, 159.94it/s]

 21%|████████████████▊                                                              | 100/471 [00:00<00:02, 158.66it/s]

 24%|███████████████████                                                            | 114/471 [00:00<00:02, 151.70it/s]

 28%|█████████████████████▉   

In [125]:
seoul_area

,emd_cd,emd_eng_nm,emd_kor_nm,geometry,car
0,11110152,Bongik-dong,봉익동,"POLYGON ((126.99254 37.57385, 126.99409 37.572...",6
1,11110153,Donui-dong,돈의동,"POLYGON ((126.99101 37.57336, 126.99178 37.570...",10
2,11560123,Mullae-dong 5(o)-ga,문래동5가,"POLYGON ((126.88453 37.51788, 126.88881 37.517...",84
3,11110141,Anguk-dong,안국동,"POLYGON ((126.98377 37.57947, 126.98426 37.579...",8
4,11110142,Sogyeok-dong,소격동,"POLYGON ((126.98112 37.58153, 126.98234 37.579...",8
...,...,...,...,...,...
462,11110145,Songhyeon-dong,송현동,"POLYGON ((126.98212 37.57863, 126.98258 37.576...",0
463,11290118,Dongseon-dong 3(sam)-ga,동선동3가,"POLYGON ((127.01958 37.59154, 127.01962 37.593...",24
464,11110123,Seorin-dong,서린동,"POLYGON ((126.98305 37.56998, 126.98285 37.568...",28
465,11110127,Gongpyeong-dong,공평동,"POLYGON ((126.98231 37.57202, 126.98429 37.572...",2


In [127]:
# 동에 대해 격자가 속하면 이를 seoul에 넣어줌
# 동 별 자동차 개수가 row가 된다.
# 그리고 격자가 동에 속하면 이 값을 격자에 넣어주면 된다. 
for i, rows in tqdm(seoul_area.iterrows(),total = seoul_area.shape[0]) : # 동 별 차량 대수
    seoul.loc[(seoul.geometry).intersects(rows.geometry),'car'] = rows['car'] + seoul['car']



  0%|                                                                                          | 0/467 [00:00<?, ?it/s]

  0%|▏                                                                                 | 1/467 [00:00<01:00,  7.71it/s]

  0%|▎                                                                                 | 2/467 [00:00<00:58,  7.93it/s]

  1%|▌                                                                                 | 3/467 [00:00<00:55,  8.38it/s]

  1%|▋                                                                                 | 4/467 [00:00<00:53,  8.68it/s]

  1%|▉                                                                                 | 5/467 [00:00<00:52,  8.83it/s]

  1%|█                                                                                 | 6/467 [00:00<00:51,  8.96it/s]

  1%|█▏                                                                                | 7/467 [00:00<00:51,  9.01it/s]

  2%|█▍                       

 14%|███████████▎                                                                     | 65/467 [00:08<00:46,  8.71it/s]

 14%|███████████▍                                                                     | 66/467 [00:08<00:49,  8.12it/s]

 14%|███████████▌                                                                     | 67/467 [00:08<01:03,  6.29it/s]

 15%|███████████▊                                                                     | 68/467 [00:08<01:06,  6.04it/s]

 15%|███████████▉                                                                     | 69/467 [00:09<01:04,  6.14it/s]

 15%|████████████▏                                                                    | 70/467 [00:09<01:03,  6.21it/s]

 15%|████████████▎                                                                    | 71/467 [00:09<01:00,  6.52it/s]

 15%|████████████▍                                                                    | 72/467 [00:09<00:57,  6.85it/s]

 16%|████████████▋              

 28%|██████████████████████                                                          | 129/467 [00:17<00:48,  7.03it/s]

 28%|██████████████████████▎                                                         | 130/467 [00:17<00:45,  7.46it/s]

 28%|██████████████████████▍                                                         | 131/467 [00:17<00:44,  7.52it/s]

 28%|██████████████████████▌                                                         | 132/467 [00:17<00:43,  7.65it/s]

 28%|██████████████████████▊                                                         | 133/467 [00:17<00:43,  7.70it/s]

 29%|██████████████████████▉                                                         | 134/467 [00:17<00:44,  7.48it/s]

 29%|███████████████████████▏                                                        | 135/467 [00:17<00:44,  7.53it/s]

 29%|███████████████████████▎                                                        | 136/467 [00:18<00:43,  7.69it/s]

 29%|███████████████████████▍   

 41%|█████████████████████████████████                                               | 193/467 [00:25<00:33,  8.29it/s]

 42%|█████████████████████████████████▏                                              | 194/467 [00:25<00:32,  8.45it/s]

 42%|█████████████████████████████████▍                                              | 195/467 [00:25<00:32,  8.32it/s]

 42%|█████████████████████████████████▌                                              | 196/467 [00:25<00:32,  8.27it/s]

 42%|█████████████████████████████████▋                                              | 197/467 [00:25<00:34,  7.81it/s]

 42%|█████████████████████████████████▉                                              | 198/467 [00:26<00:36,  7.37it/s]

 43%|██████████████████████████████████                                              | 199/467 [00:26<00:38,  6.91it/s]

 43%|██████████████████████████████████▎                                             | 200/467 [00:26<00:40,  6.52it/s]

 43%|███████████████████████████

 55%|████████████████████████████████████████████                                    | 257/467 [00:34<00:28,  7.25it/s]

 55%|████████████████████████████████████████████▏                                   | 258/467 [00:34<00:29,  7.00it/s]

 55%|████████████████████████████████████████████▎                                   | 259/467 [00:34<00:38,  5.45it/s]

 56%|████████████████████████████████████████████▌                                   | 260/467 [00:35<00:35,  5.79it/s]

 56%|████████████████████████████████████████████▋                                   | 261/467 [00:35<00:32,  6.28it/s]

 56%|████████████████████████████████████████████▉                                   | 262/467 [00:35<00:30,  6.63it/s]

 56%|█████████████████████████████████████████████                                   | 263/467 [00:35<00:28,  7.13it/s]

 57%|█████████████████████████████████████████████▏                                  | 264/467 [00:35<00:27,  7.36it/s]

 57%|███████████████████████████

 69%|██████████████████████████████████████████████████████▉                         | 321/467 [00:43<00:17,  8.25it/s]

 69%|███████████████████████████████████████████████████████▏                        | 322/467 [00:43<00:18,  7.86it/s]

 69%|███████████████████████████████████████████████████████▎                        | 323/467 [00:43<00:19,  7.45it/s]

 69%|███████████████████████████████████████████████████████▌                        | 324/467 [00:43<00:20,  6.96it/s]

 70%|███████████████████████████████████████████████████████▋                        | 325/467 [00:43<00:21,  6.74it/s]

 70%|███████████████████████████████████████████████████████▊                        | 326/467 [00:44<00:20,  6.92it/s]

 70%|████████████████████████████████████████████████████████                        | 327/467 [00:44<00:19,  7.21it/s]

 70%|████████████████████████████████████████████████████████▏                       | 328/467 [00:44<00:18,  7.32it/s]

 70%|███████████████████████████

 82%|█████████████████████████████████████████████████████████████████▉              | 385/467 [00:51<00:11,  7.22it/s]

 83%|██████████████████████████████████████████████████████████████████              | 386/467 [00:51<00:10,  7.58it/s]

 83%|██████████████████████████████████████████████████████████████████▎             | 387/467 [00:51<00:10,  7.87it/s]

 83%|██████████████████████████████████████████████████████████████████▍             | 388/467 [00:51<00:09,  8.09it/s]

 83%|██████████████████████████████████████████████████████████████████▋             | 389/467 [00:51<00:09,  8.27it/s]

 84%|██████████████████████████████████████████████████████████████████▊             | 390/467 [00:51<00:08,  8.61it/s]

 84%|██████████████████████████████████████████████████████████████████▉             | 391/467 [00:51<00:08,  8.76it/s]

 84%|███████████████████████████████████████████████████████████████████▏            | 392/467 [00:52<00:09,  8.23it/s]

 84%|███████████████████████████

 96%|████████████████████████████████████████████████████████████████████████████▉   | 449/467 [00:58<00:01,  9.10it/s]

 96%|█████████████████████████████████████████████████████████████████████████████   | 450/467 [00:59<00:01,  9.01it/s]

 97%|█████████████████████████████████████████████████████████████████████████████▎  | 451/467 [00:59<00:01,  8.92it/s]

 97%|█████████████████████████████████████████████████████████████████████████████▍  | 452/467 [00:59<00:01,  9.07it/s]

 97%|█████████████████████████████████████████████████████████████████████████████▌  | 453/467 [00:59<00:01,  8.92it/s]

 97%|█████████████████████████████████████████████████████████████████████████████▊  | 454/467 [00:59<00:01,  8.74it/s]

 97%|█████████████████████████████████████████████████████████████████████████████▉  | 455/467 [00:59<00:01,  8.62it/s]

 98%|██████████████████████████████████████████████████████████████████████████████  | 456/467 [00:59<00:01,  8.72it/s]

 98%|███████████████████████████

In [128]:
seoul.describe()

,level_0,index,val,car,car_charge
count,29339.00000,29339.000000,29339.000000,29339.000000,29339.0
mean,14669.00000,30466.356999,329.651829,2047.277651,0.0
std,8469.58411,17267.395635,270.937158,2345.645606,0.0
min,0.00000,894.000000,6.000000,0.000000,0.0
25%,7334.50000,14747.500000,128.000000,681.000000,0.0
50%,14669.00000,31665.000000,295.000000,1359.000000,0.0
75%,22003.50000,44632.500000,466.000000,2452.000000,0.0
max,29338.00000,61635.000000,13522.000000,26264.000000,0.0


In [183]:
car_charge = pd.read_excel('전기차충전소_위치(좌표포함).xlsx',encoding = 'utf-8',index_col=0)
car_charge_gdf = gpd.GeoDataFrame(car_charge, geometry=gpd.points_from_xy(car_charge['x'], car_charge['y']))
car_charge_gdf.crs = {'init': 'epsg:4326', 'no_defs': True}
car_charge_gdf = car_charge_gdf.to_crs(epsg=4326)
car_charge_gdf

## 버퍼 처리해야함

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,Company,Charge_Name,ID,Type,Region,si_gun_goo,address,y,x,geometry
0,환경부(한국자동차환경협회),종묘 공영주차장,1,DC차데모+AC3상,서울특별시,종로구,"서울특별시 종로구 종로 157 , 지하주차장 4층 하층 T구역(훈정동 1-2)",37.573134,126.994864,POINT (126.99486 37.57313)
1,환경부(한국자동차환경협회),세종로 공영주차장,1,DC차데모+AC3상+DC콤보,서울특별시,종로구,"서울특별시 종로구 세종대로 189 , 지하주차장 4층 D구역 계단실 앞(세종로 80-1)",37.573430,126.975960,POINT (126.97596 37.57343)
2,환경부(한국자동차환경협회),그랜드앰배서더 서울,1,DC차데모+AC3상+DC콤보,서울특별시,중구,"서울특별시 중구 동호로 287 , 대형버스주차장(장충동2가 186-54)",37.560579,127.002123,POINT (127.00212 37.56058)
3,환경부(한국자동차환경협회),한강진역 공영주차장,1,DC차데모+AC3상,서울특별시,용산구,"서울특별시 용산구 한남동 산10-84 , 지상실외주차장",37.539681,127.002560,POINT (127.00256 37.53968)
4,환경부(한국자동차환경협회),마장동사무소 앞(공중전화부스),1,DC차데모+AC3상+DC콤보,서울특별시,성동구,서울특별시 성동구 마장동 808 (마장동 808),37.565817,127.046113,POINT (127.04611 37.56582)
...,...,...,...,...,...,...,...,...,...,...
2087,에스트래픽,이마트 명일점,5,DC차데모+DC콤보,서울특별시,강동구,"서울특별시 강동구 고덕로 276 , 지하3층 주차장 F구역(명일동 46-4)",37.554707,127.155613,POINT (127.15561 37.55471)
2088,에스트래픽,이마트 명일점,4,DC차데모+DC콤보,서울특별시,강동구,"서울특별시 강동구 고덕로 276 , 지하3층 주차장 F구역(명일동 46-4)",37.554707,127.155613,POINT (127.15561 37.55471)
2089,에스트래픽,이마트 명일점,3,DC차데모+DC콤보,서울특별시,강동구,"서울특별시 강동구 고덕로 276 , 지하3층 주차장 F구역(명일동 46-4)",37.554707,127.155613,POINT (127.15561 37.55471)
2090,에스트래픽,이마트 명일점,2,DC차데모+DC콤보,서울특별시,강동구,"서울특별시 강동구 고덕로 276 , 지하3층 주차장 F구역(명일동 46-4)",37.554707,127.155613,POINT (127.15561 37.55471)


In [184]:
car_charge_gdf['geometry'] = car_charge_gdf.buffer(0.0005) # resolution 으로 다각형 조절 가능
car_charge_gdf.area # 버퍼 생성

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


In [188]:
# 생성한 버퍼가 coverage라고 생각하고 이에 걸치는 (intersects) 경우 car charge의 개수를 늘려준다.
for i, rows in tqdm(car_charge_gdf.iterrows(),total = car_charge_gdf.shape[0]) : # 동 별 차량 대수
    seoul.loc[(seoul.geometry).intersects(rows.geometry),'car_charge'] = seoul['car_charge'] + 1



  0%|                                                                                         | 0/2092 [00:00<?, ?it/s]

  0%|                                                                                 | 1/2092 [00:00<05:10,  6.73it/s]

  0%|                                                                                 | 2/2092 [00:00<04:46,  7.30it/s]

  0%|                                                                                 | 3/2092 [00:00<04:25,  7.86it/s]

  0%|▏                                                                                | 4/2092 [00:00<04:14,  8.22it/s]

  0%|▏                                                                                | 5/2092 [00:00<04:04,  8.53it/s]

  0%|▏                                                                                | 6/2092 [00:00<04:05,  8.50it/s]

  0%|▎                                                                                | 7/2092 [00:00<04:05,  8.50it/s]

  0%|▎                        

  3%|██▍                                                                             | 64/2092 [00:08<03:57,  8.53it/s]

  3%|██▍                                                                             | 65/2092 [00:08<03:52,  8.72it/s]

  3%|██▌                                                                             | 66/2092 [00:08<03:47,  8.90it/s]

  3%|██▌                                                                             | 67/2092 [00:08<04:08,  8.16it/s]

  3%|██▌                                                                             | 68/2092 [00:09<04:22,  7.70it/s]

  3%|██▋                                                                             | 69/2092 [00:09<04:21,  7.74it/s]

  3%|██▋                                                                             | 70/2092 [00:09<04:20,  7.77it/s]

  3%|██▋                                                                             | 71/2092 [00:09<04:19,  7.79it/s]

  3%|██▊                        

  6%|█████                                                                          | 133/2092 [00:16<03:46,  8.66it/s]

  6%|█████                                                                          | 134/2092 [00:16<03:44,  8.70it/s]

  6%|█████                                                                          | 135/2092 [00:16<03:46,  8.64it/s]

  7%|█████▏                                                                         | 136/2092 [00:17<03:39,  8.89it/s]

  7%|█████▏                                                                         | 137/2092 [00:17<03:45,  8.66it/s]

  7%|█████▏                                                                         | 138/2092 [00:17<03:44,  8.72it/s]

  7%|█████▎                                                                         | 140/2092 [00:17<03:32,  9.18it/s]

  7%|█████▎                                                                         | 141/2092 [00:17<03:40,  8.87it/s]

  7%|█████▎                     

 10%|███████▌                                                                       | 199/2092 [00:25<03:29,  9.02it/s]

 10%|███████▌                                                                       | 200/2092 [00:25<03:30,  9.00it/s]

 10%|███████▌                                                                       | 201/2092 [00:25<03:37,  8.68it/s]

 10%|███████▋                                                                       | 202/2092 [00:25<03:39,  8.62it/s]

 10%|███████▋                                                                       | 203/2092 [00:25<03:36,  8.74it/s]

 10%|███████▋                                                                       | 204/2092 [00:25<03:35,  8.76it/s]

 10%|███████▋                                                                       | 205/2092 [00:25<03:32,  8.86it/s]

 10%|███████▊                                                                       | 206/2092 [00:25<03:25,  9.16it/s]

 10%|███████▊                   

 13%|█████████▉                                                                     | 263/2092 [00:33<04:30,  6.75it/s]

 13%|█████████▉                                                                     | 264/2092 [00:33<04:26,  6.86it/s]

 13%|██████████                                                                     | 265/2092 [00:33<04:12,  7.25it/s]

 13%|██████████                                                                     | 266/2092 [00:33<03:59,  7.63it/s]

 13%|██████████                                                                     | 267/2092 [00:33<03:50,  7.91it/s]

 13%|██████████                                                                     | 268/2092 [00:33<03:39,  8.30it/s]

 13%|██████████▏                                                                    | 269/2092 [00:33<03:32,  8.57it/s]

 13%|██████████▏                                                                    | 270/2092 [00:34<03:29,  8.68it/s]

 13%|██████████▏                

...

 86%|██████████████████████████████████████████████████████████████████▉           | 1796/2092 [03:40<00:32,  9.03it/s]

 86%|███████████████████████████████████████████████████████████████████           | 1797/2092 [03:41<00:32,  9.15it/s]

 86%|███████████████████████████████████████████████████████████████████           | 1798/2092 [03:41<00:31,  9.32it/s]

 86%|███████████████████████████████████████████████████████████████████           | 1799/2092 [03:41<00:31,  9.36it/s]

 86%|███████████████████████████████████████████████████████████████████           | 1800/2092 [03:41<00:32,  9.06it/s]

 86%|███████████████████████████████████████████████████████████████████▏          | 1801/2092 [03:41<00:32,  8.91it/s]

 86%|███████████████████████████████████████████████████████████████████▏          | 1802/2092 [03:41<00:33,  8.53it/s]

 86%|███████████████████████████████████████████████████████████████████▏          | 1803/2092 [03:41<00:35,  8.15it/s]

 86%|███████████████████████████

 89%|█████████████████████████████████████████████████████████████████████▍        | 1862/2092 [03:48<00:25,  9.07it/s]

 89%|█████████████████████████████████████████████████████████████████████▍        | 1863/2092 [03:48<00:25,  8.84it/s]

 89%|█████████████████████████████████████████████████████████████████████▍        | 1864/2092 [03:49<00:26,  8.58it/s]

 89%|█████████████████████████████████████████████████████████████████████▌        | 1865/2092 [03:49<00:26,  8.67it/s]

 89%|█████████████████████████████████████████████████████████████████████▌        | 1866/2092 [03:49<00:25,  8.75it/s]

 89%|█████████████████████████████████████████████████████████████████████▌        | 1867/2092 [03:49<00:25,  8.69it/s]

 89%|█████████████████████████████████████████████████████████████████████▋        | 1868/2092 [03:49<00:28,  7.75it/s]

 89%|█████████████████████████████████████████████████████████████████████▋        | 1869/2092 [03:49<00:30,  7.29it/s]

 89%|███████████████████████████

 92%|███████████████████████████████████████████████████████████████████████▊      | 1927/2092 [03:56<00:25,  6.43it/s]

 92%|███████████████████████████████████████████████████████████████████████▉      | 1928/2092 [03:57<00:26,  6.29it/s]

 92%|███████████████████████████████████████████████████████████████████████▉      | 1929/2092 [03:57<00:25,  6.29it/s]

 92%|███████████████████████████████████████████████████████████████████████▉      | 1930/2092 [03:57<00:25,  6.38it/s]

 92%|███████████████████████████████████████████████████████████████████████▉      | 1931/2092 [03:57<00:24,  6.65it/s]

 92%|████████████████████████████████████████████████████████████████████████      | 1932/2092 [03:57<00:23,  6.78it/s]

 92%|████████████████████████████████████████████████████████████████████████      | 1933/2092 [03:57<00:22,  7.10it/s]

 92%|████████████████████████████████████████████████████████████████████████      | 1934/2092 [03:57<00:20,  7.65it/s]

 92%|███████████████████████████

 95%|██████████████████████████████████████████████████████████████████████████▏   | 1991/2092 [04:05<00:11,  8.47it/s]

 95%|██████████████████████████████████████████████████████████████████████████▎   | 1992/2092 [04:05<00:12,  8.25it/s]

 95%|██████████████████████████████████████████████████████████████████████████▎   | 1993/2092 [04:05<00:11,  8.30it/s]

 95%|██████████████████████████████████████████████████████████████████████████▎   | 1994/2092 [04:05<00:12,  8.16it/s]

 95%|██████████████████████████████████████████████████████████████████████████▍   | 1995/2092 [04:05<00:11,  8.32it/s]

 95%|██████████████████████████████████████████████████████████████████████████▍   | 1996/2092 [04:05<00:11,  8.48it/s]

 95%|██████████████████████████████████████████████████████████████████████████▍   | 1997/2092 [04:06<00:10,  8.79it/s]

 96%|██████████████████████████████████████████████████████████████████████████▍   | 1998/2092 [04:06<00:10,  8.96it/s]

 96%|███████████████████████████

 98%|████████████████████████████████████████████████████████████████████████████▋ | 2057/2092 [04:13<00:04,  8.34it/s]

 98%|████████████████████████████████████████████████████████████████████████████▋ | 2058/2092 [04:13<00:03,  8.67it/s]

 98%|████████████████████████████████████████████████████████████████████████████▊ | 2059/2092 [04:13<00:03,  8.94it/s]

 98%|████████████████████████████████████████████████████████████████████████████▊ | 2060/2092 [04:13<00:03,  8.87it/s]

 99%|████████████████████████████████████████████████████████████████████████████▊ | 2061/2092 [04:13<00:03,  8.38it/s]

 99%|████████████████████████████████████████████████████████████████████████████▉ | 2062/2092 [04:13<00:03,  8.48it/s]

 99%|████████████████████████████████████████████████████████████████████████████▉ | 2063/2092 [04:14<00:03,  8.73it/s]

 99%|████████████████████████████████████████████████████████████████████████████▉ | 2064/2092 [04:14<00:03,  8.89it/s]

 99%|███████████████████████████

In [189]:
seoul.describe()

,level_0,index,val,car,car_charge
count,29339.00000,29339.000000,29339.000000,29339.000000,29339.000000
mean,14669.00000,30466.356999,329.651829,1894.095709,0.143018
std,8469.58411,17267.395635,270.937158,2322.369666,0.798541
min,0.00000,894.000000,6.000000,0.000000,0.000000
25%,7334.50000,14747.500000,128.000000,548.000000,0.000000
50%,14669.00000,31665.000000,295.000000,1290.000000,0.000000
75%,22003.50000,44632.500000,466.000000,2325.000000,0.000000
max,29338.00000,61635.000000,13522.000000,26264.000000,71.000000


In [197]:
mart = pd.read_csv('시설/대형마트.csv', encoding = 'cp949')
mart_gdf = gpd.GeoDataFrame(mart, geometry=gpd.points_from_xy(mart['x'], mart['y']))
mart_gdf.crs = {'init': 'epsg:4326', 'no_defs': True}
mart_gdf = mart_gdf.to_crs(epsg=4326)
mart_gdf

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,id,place_name,category_name,address_name,x,y,geometry
0,25612885,홈플러스익스프레스 상계장암점,"가정,생활 > 슈퍼마켓 > 대형슈퍼 > 홈플러스익스프레스",서울 노원구 상계동 1195-1,127.052641,37.686615,POINT (127.05264 37.68661)
1,20707906,이마트에브리데이 도봉점,"가정,생활 > 슈퍼마켓 > 대형슈퍼 > 이마트 에브리데이",서울 도봉구 도봉동 62-1,127.048017,37.682570,POINT (127.04802 37.68257)
2,1216650549,조은마트 상계점,"가정,생활 > 대형마트",서울 노원구 상계동 1101-32,127.052098,37.675194,POINT (127.05210 37.67519)
3,21442742,롯데슈퍼 수락점,"가정,생활 > 슈퍼마켓 > 대형슈퍼 > 롯데슈퍼",서울 노원구 상계동 1049-71,127.054972,37.674184,POINT (127.05497 37.67418)
4,461394586,노브랜드 노원상계점,"가정,생활 > 슈퍼마켓 > 대형슈퍼 > 노브랜드",서울 노원구 상계동 1040-18,127.054226,37.670883,POINT (127.05423 37.67088)
...,...,...,...,...,...,...,...
443,8155382,홈플러스 스페셜 시흥점,"가정,생활 > 대형마트 > 홈플러스",서울 금천구 시흥동 992-47,126.900918,37.452078,POINT (126.90092 37.45208)
444,1750048518,관악농협하나로마트 시흥중앙점,"가정,생활 > 슈퍼마켓 > 대형슈퍼 > 하나로마트",서울 금천구 시흥동 936-1,126.903538,37.448184,POINT (126.90354 37.44818)
445,26912132,이마트에브리데이 내곡점,"가정,생활 > 슈퍼마켓 > 대형슈퍼 > 이마트 에브리데이",서울 서초구 신원동 203-1,127.056157,37.447165,POINT (127.05616 37.44716)
446,27273462,서울영동농협하나로마트 청계산역점,"가정,생활 > 슈퍼마켓 > 대형슈퍼 > 하나로마트",서울 서초구 신원동 202-20,127.055918,37.447664,POINT (127.05592 37.44766)


In [198]:
room = pd.read_csv('시설/숙박.csv', encoding = 'cp949')
room_gdf = gpd.GeoDataFrame(room, geometry=gpd.points_from_xy(room['x'], room['y']))
room_gdf.crs = {'init': 'epsg:4326', 'no_defs': True}
room_gdf = room_gdf.to_crs(epsg=4326)
room_gdf

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,id,place_name,category_name,address_name,x,y,geometry
0,21025433,북한산 석굴야영장,"여행 > 숙박 > 야영,캠핑장",서울 도봉구 도봉동,127.019067,37.696385,POINT (127.01907 37.69638)
1,21025395,북한산 구조대야영장09,"여행 > 숙박 > 야영,캠핑장",서울 도봉구 도봉동,127.019099,37.695833,POINT (127.01910 37.69583)
2,21025509,"북한산 구조대야영장10,11","여행 > 숙박 > 야영,캠핑장",서울 도봉구 도봉동,127.019239,37.695959,POINT (127.01924 37.69596)
3,1555779223,호텔더데이,"여행 > 숙박 > 여관,모텔",서울 도봉구 도봉동 560-16,127.045733,37.685158,POINT (127.04573 37.68516)
4,26998294,몽모텔,"여행 > 숙박 > 여관,모텔",서울 도봉구 도봉동 559-28,127.045680,37.686203,POINT (127.04568 37.68620)
...,...,...,...,...,...,...,...
3666,16768085,은성장,"여행 > 숙박 > 여관,모텔",서울 금천구 시흥동 908-22,126.906142,37.451181,POINT (126.90614 37.45118)
3667,90792441,청계산장 글램핑 (2020년 3월예정),"여행 > 숙박 > 야영,캠핑장 > 글램핑장",서울 서초구 원지동 207-1,127.046400,37.448330,POINT (127.04640 37.44833)
3668,873644582,오라카이 청계산호텔,여행 > 숙박 > 호텔,서울 서초구 신원동 271-21,127.054667,37.448978,POINT (127.05467 37.44898)
3669,16796967,아이리스모텔,"여행 > 숙박 > 여관,모텔",서울 금천구 시흥동 985-16,126.903369,37.446180,POINT (126.90337 37.44618)


In [200]:
restaurant = pd.read_csv('시설/음식점.csv', encoding = 'cp949')
restaurant_gdf = gpd.GeoDataFrame(restaurant, geometry=gpd.points_from_xy(restaurant['x'], restaurant['y']))
restaurant_gdf.crs = {'init': 'epsg:4326', 'no_defs': True}
restaurant_gdf = restaurant_gdf.to_crs(epsg=4326)
restaurant_gdf

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,id,place_name,category_name,address_name,x,y,geometry
0,8890684,도봉산해,음식점 > 한식,서울 도봉구 도봉동 349,127.041256,37.690339,POINT (127.04126 37.69034)
1,18154431,옥동송어,"음식점 > 한식 > 해물,생선 > 회",서울 도봉구 도봉동 352-38,127.041615,37.691106,POINT (127.04161 37.69111)
2,18668029,옹기꽃게장,"음식점 > 한식 > 해물,생선 > 게,대게",서울 도봉구 도봉동 286-27,127.041255,37.688501,POINT (127.04126 37.68850)
3,1013549731,더밸리,음식점 > 양식,서울 도봉구 도봉동 384-9,127.040049,37.689885,POINT (127.04005 37.68989)
4,21305968,홍두깨손칼국수,음식점 > 한식 > 국수,서울 도봉구 도봉동 286-25,127.041318,37.688212,POINT (127.04132 37.68821)
...,...,...,...,...,...,...,...
41552,8753471,고향촌돌구이,음식점 > 한식 > 한정식,서울 금천구 시흥동 977,126.904129,37.434662,POINT (126.90413 37.43466)
41553,21406411,일미식당,음식점 > 한식,서울 금천구 시흥동 970,126.904233,37.435113,POINT (126.90423 37.43511)
41554,23021590,흙돼지마을,"음식점 > 한식 > 육류,고기",서울 금천구 시흥동 978-6,126.904207,37.434305,POINT (126.90421 37.43431)
41555,14653997,닭권브이,"음식점 > 한식 > 육류,고기 > 닭요리",서울 금천구 시흥3동 975,126.906422,37.433842,POINT (126.90642 37.43384)
